In [1]:
#setup data
import pandas as pd
import numpy as np
import math
import random
from sklearn import preprocessing

ignore_features = ['25', '26', '8', '58', '20', '23', '29', '31', '32', '35']
toencode = {}
f = open("field_types.txt", 'r')
for line in f.readlines():
    line = line.strip().split(" ")
    if line[1] != "numeric" and line[0] not in ignore_features:
        toencode[line[0]] = len(line) - 1
        

f1 = open("data.csv")
data_df = pd.read_csv(f1, sep=',', header = 'infer', low_memory=False)
data_df = data_df.drop(ignore_features, axis=1)
label_df = data_df[['label']]
data_df = data_df.drop('label', axis=1)
label = np.array(label_df)


f2 = open("quiz.csv")
quiz_df = pd.read_csv(f2, sep=',', header = 'infer', low_memory=False)
quiz_df = quiz_df.drop(ignore_features, axis=1)


In [2]:
#getting test and train
data_end = np.shape(data_df)[0]
df = data_df.append(quiz_df)
df_trans = pd.get_dummies(df, columns=toencode.keys(), dummy_na=False)
data = np.array(df_trans)

data, quiz = data[:data_end], data[data_end:]

In [3]:
#splitting to features and labels by training and test
def splitdata(data, label):
    n = data.shape[0]
    train_num = int(math.ceil(n*.8))
    indices = random.sample(xrange(n), n)

    training_idx, test_idx = indices[train_num:], indices[:train_num]
    training, test = data[training_idx,:], data[test_idx,:]
    lab, testlabel = label[training_idx,:], label[test_idx,:]
    
    return training, lab, test, testlabel

train, lab, test, testlabel = splitdata(data, label)

In [4]:
#evaluator
def eval(preds, testlabels):
    if len(preds) == len(testlabels):
        error = np.count_nonzero(preds != testlabels) / np.float(len(preds))
        return 1 - error

In [5]:
#unraveling
testlabelnum = np.ravel(testlabel)
labnum = np.ravel(lab)

In [11]:
#decision tree 50 depth (0.890)
from sklearn.tree import DecisionTreeClassifier
dectree50 = DecisionTreeClassifier(max_depth=50)

ydectree50test = dectree50.fit(train, lab).predict(test)
print eval(ydectree50test, testlabelnum)

0.890322262738


In [7]:
from sklearn.linear_model import LogisticRegression

# scalar weighting
scaler = preprocessing.StandardScaler().fit(train)
trains = scaler.transform(train) 
tests = scaler.transform(test) 
quizs = scaler.transform(quiz) 

logreg = LogisticRegression()

#logistic regression (0.87656)
ylogregtest = logreg.fit(trains, lab).predict(tests)
print eval(ylogregtest, testlabelnum)

0.877550014783


/Users/radixold/anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
#adaboost on dec tree max_dept = 10 (0.9169)
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
adaboost = AdaBoostClassifier(DecisionTreeClassifier(max_depth=10),n_estimators=600, learning_rate=1)

yadaboosttest = adaboost.fit(train, lab).predict(test)
print eval(yadaboosttest, testlabelnum)

In [14]:
#adaboost on dec tree max_dept = 20 (0.9197)
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
adaboost20 = AdaBoostClassifier(DecisionTreeClassifier(max_depth=30),n_estimators=600, learning_rate=1)

yadaboosttest20 = adaboost20.fit(train, lab).predict(test)
print eval(yadaboosttest20, testlabelnum)

0.919769389967


/Users/radixold/anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [16]:
#adaboost on dec tree max_dept = 30' n_estimators 1000 (0.9203)
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
adaboost20ne = AdaBoostClassifier(DecisionTreeClassifier(max_depth=30),n_estimators=1000, learning_rate=1)

yadaboosttest20ne = adaboost20ne.fit(train, lab).predict(test)
print eval(yadaboosttest20ne, testlabelnum)

0.920370552873


/Users/radixold/anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
#adaboost on dec tree max_dept = 20' learning_rate = 2 (0.9197)
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
adaboost20lr = AdaBoostClassifier(DecisionTreeClassifier(max_depth=30),n_estimators=600, learning_rate=2)

yadaboosttest20lr = adaboost20lr.fit(train, lab).predict(test)
print eval(yadaboosttest20lr, testlabelnum)

In [14]:
#random forest classifier (0.9262)
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators=75)

yrfc = rfc.fit(train, lab).predict(test)
print eval(yrfc, testlabelnum)

0.92628363063


/Users/radixold/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


In [28]:
#extratrees on vt (0.9243)
from sklearn.ensemble import ExtraTreesClassifier
etc = ExtraTreesClassifier(n_estimators=100)

yetc = etc.fit(train, lab).predict(test)
print eval(yetc, testlabelnum)

0.924352025229


/Users/radixold/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


In [30]:
#bagging 0.9136
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
bagging = BaggingClassifier(DecisionTreeClassifier(max_depth=50))

ybagging = bagging.fit(train, lab).predict(test)
print eval(ybagging, testlabelnum)

0.9136197891


/Users/radixold/anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [31]:
#adaboost on dec tree max_depth = 50 (0.9197)
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
adaboost20 = AdaBoostClassifier(DecisionTreeClassifier(max_depth=50),n_estimators=600, learning_rate=1)

yadaboosttest20 = adaboost20.fit(train, lab).predict(test)
print eval(yadaboosttest20, testlabelnum)

0.921799546664


/Users/radixold/anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [33]:
#random forest classifier (0.9264)
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators=200)

yrfc = rfc.fit(train, lab).predict(test)
print eval(yrfc, testlabelnum)

0.926470878092


/Users/radixold/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


In [8]:
#random forest classifier (0.9269)
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators=3000)

yrfc = rfc.fit(train, lab).predict(test)
print eval(yrfc, testlabelnum)

0.927062185868


/Users/radixold/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


In [9]:
yrfc

array([-1, -1, -1, ..., -1, -1, -1])

In [20]:
classifier = []
classifier.append(quizrfc)
classifier_df = pd.DataFrame(classifier)
classifier_df = classifier_df.T
classifier_df.to_csv('out.csv', index=False)

In [19]:
quizrfc = rfc.fit(train, lab).predict(quiz)

/Users/radixold/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  if __name__ == '__main__':
